In [ ]:
from __future__ import unicode_literals

import sys, os
BIN = os.path.expanduser("../../")
sys.path.append(BIN)

import numpy as np
from scipy.constants import m_p, c, e, pi
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import itertools

from test_tools import generate_objects, BunchTracker, track, compare_traces, compare_projections, Machine

from PyHEADTAIL_feedback.feedback import OneboxFeedback, Kicker, PickUp
from PyHEADTAIL_feedback.processors import ChargeWeighter, Averager, Bypass
from PyHEADTAIL_feedback.digital_processors import HilbertPhaseShiftRegister

np.random.seed(0)

In [ ]:
""" 
   This is an example of utilizing  . There are general rule for usage 
   
   The example is 
"""

In [ ]:
# basic parameters of the simulation
n_macroparticles = 10000
n_slices = 20
n_segments = 5
n_sigma_z = 3 

n_turns = 400

bunch_ref, slicer_ref,trans_map, long_map = generate_objects(n_macroparticles,n_segments, n_slices,n_sigma_z)
bunch_unkicked = copy.deepcopy(bunch_ref)


In [ ]:
# this creates an artificially kicked bunch, which will be damped with different feedback systems

slice_set = bunch_ref.get_slices(slicer_ref, statistics=True)
p_idx = slice_set.particles_within_cuts
s_idx = slice_set.slice_index_of_particle.take(p_idx)

# random kicks
kick_x = 0.003*(-1.0+2*np.random.rand(n_slices))
kick_y = 0.003*(-1.0+2*np.random.rand(n_slices))

for p_id, s_id in itertools.izip(p_idx,s_idx):
    bunch_ref.x[p_id] += kick_x[s_id]
    bunch_ref.y[p_id] += kick_y[s_id]
    
bunch_init = copy.deepcopy(bunch_ref)
tracker_ref = BunchTracker(bunch_init)
maps_ref = [i for i in trans_map] + [long_map]
track(n_turns, bunch_init,maps_ref,tracker_ref)

# The machine parameters have been determined in Machine() object in file test_tools.py
machine = Machine()

In [ ]:
# In the used code, the parameter 'gain' is determined as a fraction of the input signal if the used signal 
# processors would bypass the signalperfectly

# There are two ways to set a gain value. If only one value is given, same value will be used both
# in horizontal and vertical plane. If two values are given, separed values (x,y) are used for horizontal and
# vertical planes

feedback_gain = 0.01
# feedback_gain = (0.01,0.01)

delay = 1 
n_values = 3

In [ ]:
# As it was explained, registers are used for storing the signal and rotating it in the betatron phase. Thus,
# a signal recorded in one place of the accelerator (e.g. pickup) can be rotated to betatron pahse in another 
# place of the accelerator (e.g. kicker) by using a register
#
# A feedback system consisting of a separated pickup and kicker can be implemented very similarly to the case
# of OneboxFeedback. Instead of adding OneboxFeedback object to the total tracking map of PyHEADTAIL, 
# separated Pickup and Kicker objects are added to correct positions of the map. The signal in both pickup 
# and kicker can be processed by giving a list of signal processors as a parameter to the pickup and kicker. 
# The signal from the pickup is transferred to the kicker by giving a reference to the register, which is 
# a part of the signal processors of the pickup, as an input parameter to the kicker. 

bunch_separated_example = copy.deepcopy(bunch_ref)
tracker_separated_example = BunchTracker(bunch_separated_example)
slicer_separated_example = copy.deepcopy(slicer_ref)




# At first, the pickup is created. It uses exactly same signal processors as in the previous example.
processors_separated_pickup_x = [
    ChargeWeighter(normalization = 'average_weight'),
    Averager(),
    HilbertPhaseShiftRegister(n_values, machine.Q_x, delay)
]
processors_separated_pickup_y = [
    ChargeWeighter(normalization = 'average_weight'),
    Averager(),
    HilbertPhaseShiftRegister(n_values, machine.Q_y, delay)
]

# A position of the kicker in units of betatron phase advance, must be given as a input parameter for 
# Pickup(...) object. The total betatron phase angle over the accelerator is 2*pi*Q. In this example, 
# the accelerator is divided into a number of equally length segments determined by 'n_segments'. Thus, 
# a length of the one segment in betatron phase is 2*pi*Q/n_segments. The location of the pickup is after 
# the first segment, so this number is multiplied by one, i.e. 
pickup_position_x = 1.*2.*pi/float(n_segments)*machine.Q_x
pickup_position_y = 1.*2.*pi/float(n_segments)*machine.Q_y

# The map element is created by giving a slicer object and the parameters defined above as input parameters
pickup_map = PickUp(slicer_separated_example,processors_separated_pickup_x,processors_separated_pickup_y, 
       pickup_position_x, pickup_position_y)




# The kicker can be created very similarly to the pickup. In addition to the input parameters given to PickUp,
# a gain value, references to the register elements in signal processors of the pickup and conversion 
# coefficients for amplitude conversion from displacement to divergence must be given as input parameters.

# In this example, signal processing in the kicker is not needed, so Bypass() processors are used
processors_separated_kicker_x = [Bypass()]
processors_separated_kicker_y = [Bypass()]

# In here a list of register element(s) of the pickup(s) is created
registers_x = [processors_separated_pickup_x[2]]
registers_y = [processors_separated_pickup_y[2]]

# In this example, the position of the register is after the second map element of of the trans_map.
# In betatron phase advance it means that
kicker_position_x = 2.*2.*pi/float(n_segments)*machine.Q_x
kicker_position_y = 2.*2.*pi/float(n_segments)*machine.Q_y

# The signal units in a pickup and a kicker are different. The pickup measures displacement in the units of 
# distance [m], but kick is given in the kicker in the units of divergence [radian]. Thus, conversion
# coefficients for units must given for kicker. They can be get, for example, from the ratios of sigma values
xp_per_x = bunch_unkicked.sigma_xp()/bunch_unkicked.sigma_x()
yp_per_y = bunch_unkicked.sigma_yp()/bunch_unkicked.sigma_y()

kicker_map = Kicker(feedback_gain, slicer_separated_example,
                    processors_separated_kicker_x, processors_separated_kicker_y,
                    kicker_position_x, kicker_position_y,
                    registers_x, registers_y, xp_per_x, yp_per_y)

# After that the maps must be added to correct slots of the total map determined by the positions of 
# the picup and the kicker
total_map_separated_example = [trans_map[0]] + [pickup_map] + [trans_map[1]] + [kicker_map]
for element in trans_map[2:]:
    total_map_separated_example += [element]
total_map_separated_example += [long_map]
    
    
track(n_turns, bunch_separated_example,total_map_separated_example,tracker_separated_example)

In [ ]:
# In here, the traces and the projections from different implementations of the feedback system are compared.
# Note the scale in the emittance figures.

compare_traces([tracker_OneBox_bunch,tracker_register_example,tracker_separated_example],
               ['Ideal', 'Delayed', 'Separated'])
compare_projections([ bunch_OneBox_bunch,  bunch_register_example, bunch_separated_example], 
                    ['Ideal', 'Delayed', 'Separated'])